https://huggingface.co/docs/transformers/tasks/language_modeling
Link to follow the tutorial

In [1]:
import pandas as pd
import numpy as np
import gc
from transformers import Adafactor
import tqdm
from tqdm.notebook import tqdm_notebook
import torch
import evaluate
import wandb
import math

In [2]:
wandb.init(project="T5-Custom-Tokenizer", resume=True)

wandb: Currently logged in as: fayz. Use `wandb login --relogin` to force relogin


In [3]:
if torch.cuda.is_available():
   dev = torch.device("cuda:0")
   print("Running on the GPU")
else:
   dev = torch.device("cpu")
   print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_json('/home/beyond-data/Documents/Faiza/Notebooks for Finetuning/Data/train.json')
test = pd.read_json('/home/beyond-data/Documents/Faiza/Notebooks for Finetuning/Data/test.json')
result_df = pd.read_csv('Custom_Tokenizer/results.csv')

epoch = len(result_df['epoch'])+1

train['input'] = train['Question'] + train['document']
test['input'] = test['Question'] + test['document']
del train['Question']
del test['Question']
del train['document']
del test['document']
#trsin
for i in tqdm_notebook(range(len(train['input']))):
  if len(train['input'][i]) > 512:
    train['input'][i] = train['input'][i][:512]
for i in tqdm_notebook(range(len(train['Answer']))):
  if len(train['Answer'][i]) > 512:
    train['Answer'][i] = train['Answer'][i][:512]
#test
for i in tqdm_notebook(range(len(test['input']))):
  if len(test['input'][i]) > 512:
    test['input'][i] = test['input'][i][:512]
for i in tqdm_notebook(range(len(test['Answer']))):
  if len(test['Answer'][i]) > 512:
    test['Answer'][i] = test['Answer'][i][:512]

  0%|          | 0/46753 [00:00<?, ?it/s]

  0%|          | 0/46753 [00:00<?, ?it/s]

  0%|          | 0/6676 [00:00<?, ?it/s]

  0%|          | 0/6676 [00:00<?, ?it/s]

In [5]:
result_df

,epoch,training_loss,val_loss,ppl


In [ ]:
del result_df['Unnamed: 0']

In [6]:
len(result_df['epoch'])

0

In [ ]:
print("T5forQAS"+str(epoch-1))

### Pre-trained checkpoint from hugging face

In [7]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration
tokenizer = T5TokenizerFast.from_pretrained('T5tokenizer')
if len(result_df['epoch'])>0:
    model = T5ForConditionalGeneration.from_pretrained("Custom_Tokenizer/T5forQAS"+str(epoch-1)).to(dev)
else:
    model = T5ForConditionalGeneration.from_pretrained("t5-base").to(dev)

seed = 1337
np.random.seed(seed)
#random.seed(52)
train_df=train
batch_size=6
num_of_batches=len(train_df)/batch_size
print(num_of_batches)
gc.collect()

optimizer = Adafactor(model.parameters(),lr=0.001,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

from IPython.display import HTML, display
def progress(loss,value, max=100):
 return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

7792.166666666667


In [ ]:
epoch

In [8]:
def model_validation(model,epoch, running_loss):
    model.eval()
    bs = 8
    val_loss=0
    ppl=0
    test_num_of_batches=len(test)/bs
    predictions = []
    torch.cuda.empty_cache()
    gc.collect()
    print("Validating the model")
    for i in tqdm_notebook(range(int(test_num_of_batches))):
        ib=[]
        lb = []
        new_df=test[i*bs:i*bs+bs]
        for indx,row in new_df.iterrows():
          input =row['input']
          ib.append(input)
          lb.append(row['Answer'])
          torch.cuda.empty_cache()
        ib=tokenizer.batch_encode_plus(ib,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"].to(dev)
        lb=tokenizer.batch_encode_plus(lb,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"].to(dev)
        optimizer.zero_grad()
        outputs = model(input_ids=ib,labels=lb)
        loss = outputs.loss
        val_loss += loss.item()
    val_loss = val_loss/test_num_of_batches
    print("Validation loss: ", val_loss)
    wandb.log({"Training loss":running_loss,"Validation loss": val_loss})
    try:
        val_loss = np.mean(val_loss)
        ppl = math.exp(val_loss)
        wandb.log({"Perplexity": ppl})
    except:
        pass
    result_row = []
    result_row.append(epoch)
    result_row.append(running_loss)
    result_row.append(val_loss)
    result_row.append(ppl)
    result_df.loc[len(result_df)]=result_row
    result_df.to_csv('Custom_Tokenizer/results.csv')

In [9]:
#from os import truncate
model.train()

loss_per_10_steps=[]
num_of_epochs=30
for epoch in range(epoch,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in tqdm_notebook(range(int(num_of_batches))):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'LFQA: '+row['input']+'</s>' 
      labels = row['Answer']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
      torch.cuda.empty_cache()
    inputbatch=tokenizer.batch_encode_plus(inputbatch,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,truncation = True,padding=True,max_length=512,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    #torch.empty_cache()
    gc.collect()
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  #wandb.log({"Training loss": running_loss})
  wandb.watch(model)
  model.save_pretrained('Custom_Tokenizer/T5forQAS'+str(epoch))
  model_validation(model, epoch, running_loss)

Running epoch: 1


  0%|          | 0/7792 [00:00<?, ?it/s]

Epoch: 1 , Running loss: 3.136105124817614
Validating the model


  0%|          | 0/834 [00:00<?, ?it/s]

Validation loss:  3.4671628708036595
Running epoch: 2


...

wandb: ERROR Metric data exceeds maximum size of 10.4MB (14.5MB)


Epoch: 30 , Running loss: 0.12428539507656837
Validating the model


  0%|          | 0/834 [00:00<?, ?it/s]

Validation loss:  6.226001144241187


wandb: ERROR Metric data exceeds maximum size of 10.4MB (15.0MB)


In [ ]:
model

In [ ]:
#from os import truncate
model.train()

loss_per_10_steps=[]
num_of_epochs=40
for epoch in range(epoch,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in tqdm_notebook(range(int(num_of_batches))):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'LFQA: '+row['input']+'</s>' 
      labels = row['Answer']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
      torch.cuda.empty_cache()
    inputbatch=tokenizer.batch_encode_plus(inputbatch,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,truncation = True,padding=True,max_length=512,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    #torch.empty_cache()
    gc.collect()
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  wandb.log({"Training loss": running_loss})
  wandb.watch(model)
  model.save_pretrained('T5forQASbase'+str(epoch))
  model_validation(model, epoch, running_loss)

In [ ]:
model_validation(model, epoch, 2.58471687418077)

In [ ]:
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
def model_validation(model,epoch, running_loss):
    model.eval()
    bs = 32
    test_num_of_batches=len(test)/bs
    predictions = []
    torch.cuda.empty_cache()
    gc.collect()
    print("Validating the model")
    for i in tqdm_notebook(range(int(test_num_of_batches))):
        ib=[]
        lb = []
        new_df=test[i*bs:i*bs+bs]
        for indx,row in new_df.iterrows():
          input =row['input']
          ib.append(input)
          lb.append(row['Answer'])
          torch.cuda.empty_cache()
        ib=tokenizer.batch_encode_plus(ib,truncation=True,padding=True,max_length=512,return_tensors='pt')["input_ids"].to(dev)
        outputs = model.generate(ib, do_sample=False,max_length=512,
                            min_length=50,num_return_sequences=1)
        preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for p in preds:
            predictions.append(p)
        torch.cuda.empty_cache()
        gc.collect()
    #rouge = evaluate.load('rouge')
    print(len(predictions))
    references = list(test['Answer'][:len(predictions)])
    new_ref = []
    new_pred = []
    for i in range(len(test['Answer'][:len(predictions)])):
        if(len(test['Answer'][i]) != 0):
            new_ref.append(test['Answer'][i])
            new_pred.append(predictions[i])
    print(len(new_ref))
    print(len(new_pred))
    results = rouge.get_scores(new_pred,new_ref, avg=True)
    print(results)
#     m_p_r1 = 0
#     m_r_r1 = 0
#     result_row = []
#     result_row.append(running_loss)
#     result_row.append(epoch)
#     for k in results.keys():
#         p=[]
#         r=[]
#         f=[]
#         for a in range(len(results[k])):
#             p.append(results[k][a][0])
#             r.append(results[k][a][1])
#             f.append(results[k][a][2])
#         m_p_r1 = sum(p)/len(p)
#         m_r_r1 = sum(r)/len(r)
#         m_f_r1 = sum(f)/len(f)
#         print(m_p_r1, '-------',m_r_r1,'-------',m_f_r1)
#         result_row.append(m_p_r1)
#         result_row.append(m_r_r1)
#         result_row.append(m_f_r1)
#     result_df.loc[len(result_df)]=result_row[:11]
#     result_df.to_csv("T5result.csv")

In [ ]:
new_ref = []
new_pred = []
for i in range(len(test['Answer'])):
    if(len(test['Answer'][i]) != 0):
        new_ref.append(test['Answer'][i])
        new_pred.append(predictions[i])

### Results after 4 epochs
                        Precision                   Recall                       F1
    Rouge1:      0.4980928630793733 ------- 0.13011565464471825 ------- 0.1649583209910756
    Rouge2:      0.21678062791504746 ------- 0.06789696353215356 ------- 0.08347085728346133
    RougeL:      0.42345051832613684 ------- 0.11469741656509573 ------- 0.14313986850382462
    RougeLsum:   0.42345051832613684 ------- 0.11469741656509573 ------- 0.14313986850382462
    
    Running Loss at 5 epochs: 1.22
    
### Results after 10 epochs

                     Precision                   Recall                       F1
    Rouge1:      0.38907256490037506 ------- 0.20799839653432498 ------- 0.20409757332408776
    Rouge2:      0.1330397499564915 ------- 0.08199202715107086 ------- 0.07403134869539121
    RougeL:      0.29096532928037117 ------- 0.1632024864354101 ------- 0.1549482945224556
    RougeLsum:   0.29096532928037117 ------- 0.1632024864354101 ------- 0.1549482945224556
    
    Running Loss at 5 epochs: 0.965
    
### Results after 19 epochs

                   Precision                   Recall                       F1
    Rouge1:      0.39236778993746196 ------- 0.2466480379654679 ------- 0.23611925653365817
    Rouge2:      0.13701988445759425 ------- 0.09518066010175917 ------- 0.08576509087949875
    RougeL:      0.2764487873346976 ------- 0.1822455615208833 ------- 0.1686459668576014
    RougeLsum:   0.2764487873346976 ------- 0.1822455615208833 ------- 0.1686459668576014
    
    Running Loss at 5 epochs: 0.737
    
 ### Results after 30 epochs
 
                 Precision                   Recall                       F1
    Rouge1:      0.39773214385507505 ------- 0.20768996571589074 ------- 0.20621252890017672
    Rouge2:      0.12562862373620837 ------- 0.08111509973311858 ------- 0.07048492927928829
    RougeL:      0.2855979383154639 ------- 0.15646543259919732 ------- 0.14925823758712192
    RougeLsum:   0.2855979383154639 ------- 0.15646543259919732 ------- 0.14925823758712192
    
    Running Loss at 5 epochs: 0.51
    
    
### Results after 40 epochs
   
                  Precision                   Recall                       F1
    Rouge1:      0.28412939310936774 ------- 0.29322542427722015 ------- 0.25636404308564037
    Rouge2:      0.07850300360333388 ------- 0.09968486634318151 ------- 0.07341643688326621
    RougeL:      0.19592878386406765 ------- 0.210215201335104 ------- 0.1770061998477224
    RougeLsum:   0.19592878386406765 ------- 0.210215201335104 ------- 0.1770061998477224
    
    Running Loss at 5 epochs: 0.362


In [ ]:
model